In [1]:
from google.colab import drive
from google.colab import auth
from google.auth import default
import os

# Attempt to unmount the drive if it's already mounted and remove the directory
try:
  drive.flush_and_unmount()
except ValueError:
  pass # Drive was not mounted, continue

# Explicitly remove and recreate the mount point directory
if os.path.exists('/content/drive/'):
    os.system('rm -rf /content/drive/')
os.makedirs('/content/drive/', exist_ok=True)

drive.mount('/content/drive/', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive/


In [2]:
import os, pandas as pd, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# paths / hyperparams
CSV  = "/content/drive/MyDrive/HyacinthWatch_workspace/HyacinthWatch_data/processed/metadata.csv"
RUNS = "/content/drive/MyDrive/HyacinthWatch_workspace/HyacinthWatch_data/runs"
SIZE, BATCH, EPOCHS, LR = 224, 32, 5, 1e-3

os.makedirs(RUNS, exist_ok=True)


# filter Bangladesh augmented only
df = pd.read_csv(CSV)
train_df = df.query("source=='bangladesh_augmented' and split=='train'")
val_df   = df.query("source=='bangladesh_augmented' and split=='val'")
print(len(train_df), "train,", len(val_df), "val")  # quick sanity

2603 train, 643 val


Dataset & Loaders

In [3]:
import numpy as np

class PresenceDataset(Dataset):
    def __init__(self, frame, size=224):
        self.df = frame.reset_index(drop=True); self.size = size
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        x = Image.open(r.image_path).convert("RGB").resize((self.size,self.size))
        x = (np.asarray(x).transpose(2,0,1)/255.0).astype("float32")
        y = np.float32(r.has_hyacinth)
        return x, y

dl_tr = DataLoader(PresenceDataset(train_df, SIZE), batch_size=BATCH, shuffle=True,  num_workers=2)
dl_va = DataLoader(PresenceDataset(val_df,   SIZE), batch_size=BATCH*2, shuffle=False, num_workers=2)

Model, Loss, Train Loop (MobileNetV2 head)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
backbone = torch.hub.load('pytorch/vision:v0.14.1','mobilenet_v2', pretrained=True)
backbone.classifier[1] = nn.Linear(backbone.last_channel, 1)
model = backbone.to(device)

loss_fn = nn.BCEWithLogitsLoss()
opt = optim.Adam(model.parameters(), lr=LR)

def run_epoch(dl, train=True):
    model.train(train)
    total = 0.0; n=0
    with torch.set_grad_enabled(train):
        for xb, yb in dl:
            xb = torch.tensor(xb, device=device)
            yb = torch.tensor(yb, device=device).float().unsqueeze(1)
            if train: opt.zero_grad()
            logits = model(xb)
            loss = loss_fn(logits, yb)
            if train: loss.backward(); opt.step()
            total += loss.item()*xb.size(0); n += xb.size(0)
    return total/max(n,1)

best = 1e9
for ep in range(EPOCHS):
    tr = run_epoch(dl_tr, True)
    va = run_epoch(dl_va, False)
    print(f"epoch {ep+1}: train {tr:.4f}  val {va:.4f}", flush=True)
    if va < best:
        best = va
        torch.save(model.state_dict(), f"{RUNS}/presence_mobilenetv2.pt")
print("saved best to", f"{RUNS}/presence_mobilenetv2.pt")

Downloading: "https://github.com/pytorch/vision/zipball/v0.14.1" to /root/.cache/torch/hub/v0.14.1.zip


/root/.cache/torch/hub/pytorch_vision_v0.14.1/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
/root/.cache/torch/hub/pytorch_vision_v0.14.1/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/.cache/torch/hub/pytorch_vision_v0.14.1/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 312MB/s]
/tmp/ipython-input-2409567763.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xb = torch.tensor(xb, device=device)
/tmp/ipython-input-2409567763.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yb = torch.tensor(yb, device=device).float().unsqueeze(1)


epoch 1: train 0.1439  val 0.0761
epoch 2: train 0.0622  val 0.0316
epoch 3: train 0.0464  val 0.0344
epoch 4: train 0.0137  val 0.0180
epoch 5: train 0.0230  val 0.1401
saved best to /content/drive/MyDrive/HyacinthWatch_workspace/HyacinthWatch_data/runs/presence_mobilenetv2.pt


Quick Metrics (acc, confusion matrix)

In [4]:
import torch, numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Load the best model state dictionary
model.load_state_dict(torch.load(f"{RUNS}/presence_mobilenetv2.pt", map_location=device))
model.eval()
ys, ps = [], []
with torch.no_grad():
    for xb, yb in dl_va:
        # Ensure xb is a tensor and on the correct device
        if not isinstance(xb, torch.Tensor):
            xb = torch.tensor(xb, device=device)
        else:
            xb = xb.to(device)

        logits = model(xb)
        prob = torch.sigmoid(logits).squeeze(1).cpu().numpy()
        ps.extend((prob>0.5).astype(np.uint8).tolist())
        ys.extend(yb.cpu().numpy().astype(np.uint8).tolist())


# compute confusion matrix
cm = confusion_matrix(ys, ps)
acc = accuracy_score(ys, ps)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Non-Hyacinth", "Hyacinth"])

fig, ax = plt.subplots(figsize=(5,4))
disp.plot(cmap="Greens", ax=ax)
plt.title(f"Presence Classifier Confusion Matrix\nAccuracy = {acc*100:.1f}%")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.show()

NameError: name 'model' is not defined